# 🧠 Financial Sentiment Model Explainability Dashboard

## Overview
This notebook provides comprehensive explainability analysis for the fine-tuned TinyBERT financial sentiment classification model. It includes four complementary explanation methods accessible through an interactive dashboard.

### Explanation Methods
- **🎯 SHAP**: Game-theory based feature importance
- **🔍 LIME**: Local interpretable model-agnostic explanations 
- **👁️ Attention**: Model attention head visualization
- **🌡️ GradCAM**: Gradient-based visual attribution

### Dashboard Features
- **Mistake Analysis**: Examine specific model errors
- **Custom Text Analysis**: Test any financial text
- **Interactive Interface**: Tabbed layout for easy comparison
- **On-demand Computation**: Optimized performance

## 1. 📦 Setup & Imports

In [1]:
!ls

0_setup.ipynb                  5_explainability.ipynb
1_data_processing.ipynb        5_explainability_backup.ipynb
2_train_models.ipynb           5_explainability_clean.ipynb
3_convert_to_onnx.ipynb        5_explainability_minimal.ipynb
4_benchmarks.ipynb             6_fine_tune.ipynb


In [2]:
%cd ..

/Users/matthew/Documents/deepmind_internship


In [3]:
# Core libraries
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Model and tokenizer
from transformers import BertTokenizerFast, BertForSequenceClassification
from sklearn.preprocessing import LabelEncoder

# Explainability libraries
import shap
from lime.lime_text import LimeTextExplainer
from bertviz import head_view
from captum.attr import LayerGradCam

# Dashboard components
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


## 2. 🗂️ Data & Model Loading

In [4]:
# Configuration
MODEL_DIR = Path('models/tinybert-financial-classifier')
DATA_FILE = 'data/FinancialPhraseBank/all-data.csv'
RANDOM_SEED = 42
TEST_SIZE = 0.25

# Load full dataset and create train-test split
from sklearn.model_selection import train_test_split

# Load data with correct encoding and column names (matching training notebook)
df = pd.read_csv(DATA_FILE, header=None, names=["label", "sentence"], encoding="latin-1")
df["sentence"] = df["sentence"].str.strip('"')  # Remove extra quotes

# Create train-test split with same parameters as training
train_df, test_df = train_test_split(
    df, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_SEED, 
    stratify=df['label']
)

# Extract test data
test_texts = test_df['sentence'].tolist()  # Note: column is 'sentence' not 'text'

# Load label encoder
import pickle
with open(MODEL_DIR / 'label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

true_labels_encoded = label_encoder.transform(test_df['label'])

print(f"📊 Loaded full dataset: {len(df)} samples")
print(f"📊 Test set: {len(test_texts)} samples (25% split)")
print(f"📋 Label classes: {list(label_encoder.classes_)}")
print(f"🎲 Random seed: {RANDOM_SEED}")
print(f"✅ Data loaded successfully with correct encoding")

📊 Loaded full dataset: 4846 samples
📊 Test set: 1212 samples (25% split)
📋 Label classes: ['negative', 'neutral', 'positive']
🎲 Random seed: 42
✅ Data loaded successfully with correct encoding


In [5]:
# Load model and tokenizer
print("🔄 Loading model and tokenizer...")

tokenizer = BertTokenizerFast.from_pretrained(MODEL_DIR)
pt_model = BertForSequenceClassification.from_pretrained(MODEL_DIR)
pt_model.eval()

print("✅ Model and tokenizer loaded successfully")
print(f"📱 Model type: {type(pt_model).__name__}")
print(f"🎯 Number of classes: {pt_model.config.num_labels}")

🔄 Loading model and tokenizer...
✅ Model and tokenizer loaded successfully
📱 Model type: BertForSequenceClassification
🎯 Number of classes: 3


## 3. 🔧 Core Prediction Functions

In [6]:
def predict_class(texts):
    """Predict sentiment class for text(s)"""
    if isinstance(texts, str):
        texts = [texts]
    
    predictions = []
    pt_model.eval()
    
    with torch.no_grad():
        for text in texts:
            encoding = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)
            outputs = pt_model(**encoding)
            predicted_class = torch.argmax(outputs.logits, dim=-1).item()
            predictions.append(predicted_class)
    
    return np.array(predictions)

def predict_probs_for_shap(texts):
    """Get prediction probabilities for SHAP"""
    if isinstance(texts, str):
        texts = [texts]
    
    all_probs = []
    pt_model.eval()
    
    with torch.no_grad():
        for text in texts:
            encoding = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)
            outputs = pt_model(**encoding)
            probs = torch.softmax(outputs.logits, dim=-1).squeeze().numpy()
            all_probs.append(probs)
    
    return np.array(all_probs)

def predict_probs_for_lime(texts):
    """Get prediction probabilities for LIME (expects different format)"""
    if isinstance(texts, str):
        texts = [texts]
    elif isinstance(texts, list) and len(texts) == 1 and isinstance(texts[0], str):
        # LIME sometimes passes single text as list
        texts = texts
    elif isinstance(texts, (list, tuple)) and all(isinstance(t, str) for t in texts):
        # LIME passes list of texts
        texts = list(texts)
    else:
        # Convert to list if needed
        texts = [str(t) for t in texts]
    
    all_probs = []
    pt_model.eval()
    
    with torch.no_grad():
        for text in texts:
            try:
                encoding = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)
                outputs = pt_model(**encoding)
                probs = torch.softmax(outputs.logits, dim=-1).squeeze().cpu().numpy()
                all_probs.append(probs)
            except Exception as e:
                print(f"Error processing text: {text[:50]}... Error: {e}")
                # Return default probabilities if processing fails
                all_probs.append(np.array([0.33, 0.33, 0.34]))
    
    return np.array(all_probs)

print("✅ Core prediction functions defined")

✅ Core prediction functions defined


## 4. 🧩 Explainability Methods

### 4.1 SHAP Implementation

In [7]:
# SHAP explainer (lazy initialization for performance)
_shap_explainer = None

def get_shap_explainer():
    """Get SHAP explainer (lazy initialization)"""
    global _shap_explainer
    if _shap_explainer is None:
        print("🧮 Initializing SHAP explainer...")
        _shap_explainer = shap.Explainer(predict_probs_for_shap, tokenizer)
    return _shap_explainer

def explain_with_shap(text, target_class=None):
    """Generate SHAP explanation for text"""
    print("⏳ Computing SHAP values...")
    
    explainer = get_shap_explainer()
    shap_values = explainer([text])
    
    if target_class is None:
        target_class = predict_class(text)[0]
    
    # Display SHAP plot
    shap.plots.text(shap_values[0, :, target_class])
    
    pred_label = label_encoder.inverse_transform([target_class])[0]
    print(f"📊 SHAP explanation for class: {pred_label}")

print("✅ SHAP implementation ready")

✅ SHAP implementation ready


### 4.2 LIME Implementation

In [8]:
# LIME explainer (lazy initialization)
_lime_explainer = None

def get_lime_explainer():
    """Get LIME explainer (lazy initialization)"""
    global _lime_explainer
    if _lime_explainer is None:
        _lime_explainer = LimeTextExplainer(
            class_names=label_encoder.classes_
            # Removed 'mode' parameter as it's not valid for LimeTextExplainer
        )
    return _lime_explainer

def explain_with_lime(text):
    """Generate LIME explanation for text"""
    print("⏳ Computing LIME explanation...")
    
    explainer = get_lime_explainer()
    explanation = explainer.explain_instance(
        text,
        predict_probs_for_lime,
        num_features=20,
        labels=(0, 1, 2)
    )
    
    display(HTML(explanation.as_html()))
    print("📊 LIME explanation generated")

print("✅ LIME implementation ready")

✅ LIME implementation ready


### 4.3 Attention Visualization

In [9]:
def explain_with_attention(text):
    """Generate attention visualization for text"""
    print("⏳ Generating attention visualization...")
    
    try:
        # Tokenize with attention output
        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
        
        # Get model outputs with attention
        with torch.no_grad():
            # Force eager attention for BertViz compatibility
            original_impl = getattr(pt_model.config, '_attn_implementation', None)
            pt_model.config._attn_implementation = 'eager'
            
            outputs = pt_model(**inputs, output_attentions=True)
            attentions = outputs.attentions
            tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
            
            # Restore original implementation
            if original_impl is not None:
                pt_model.config._attn_implementation = original_impl
        
        # Check if we have valid attention and tokens
        if attentions is None or len(attentions) == 0:
            print("❌ No attention weights available")
            return
            
        if len(tokens) == 0:
            print("❌ No tokens available")
            return
        
        pred_class = torch.argmax(outputs.logits, dim=-1).item()
        pred_label = label_encoder.inverse_transform([pred_class])[0]
        
        # Try BertViz first
        try:
            # Enable widget display
            from IPython.display import Javascript
            display(Javascript("""
                require.config({
                    paths: {
                        d3: 'https://d3js.org/d3.v5.min'
                    }
                });
            """))
            
            print("🎯 Attempting interactive attention visualization...")
            head_view(attentions, tokens)
            print(f"👁️ Interactive attention visualization for prediction: {pred_label}")
            
        except Exception as viz_error:
            print(f"❌ BertViz interactive view failed: {viz_error}")
            print("💡 Using custom attention heatmap...")
            
            # Custom attention visualization
            _visualize_attention_heatmap(attentions, tokens, pred_label)
            
    except Exception as e:
        print(f"❌ Attention analysis failed: {str(e)}")
        print("💡 This might be due to model architecture or BertViz compatibility issues")

def _visualize_attention_heatmap(attentions, tokens, pred_label):
    """Create custom attention heatmap visualization"""
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Get average attention across all layers and heads
    # Shape: (num_layers, batch_size, num_heads, seq_len, seq_len)
    avg_attention = torch.stack(attentions).mean(dim=0)  # Average across layers
    avg_attention = avg_attention.mean(dim=1)  # Average across heads
    attention_matrix = avg_attention[0].detach().cpu().numpy()  # Get first (and only) batch
    
    # Clean tokens for display
    clean_tokens = []
    for token in tokens:
        if token.startswith('##'):
            clean_tokens.append(token[2:])
        elif token in ['[CLS]', '[SEP]', '[PAD]']:
            clean_tokens.append(token)
        else:
            clean_tokens.append(token)
    
    # Limit to reasonable size for visualization
    max_len = min(len(clean_tokens), 50)
    attention_matrix = attention_matrix[:max_len, :max_len]
    display_tokens = clean_tokens[:max_len]
    
    # Create visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    # 1. Full attention heatmap
    sns.heatmap(attention_matrix, 
                xticklabels=display_tokens,
                yticklabels=display_tokens,
                cmap='Blues',
                ax=ax1,
                cbar_kws={'label': 'Attention Weight'})
    ax1.set_title(f'Attention Heatmap\nPrediction: {pred_label}', fontsize=14, weight='bold')
    ax1.set_xlabel('Attended Tokens')
    ax1.set_ylabel('Query Tokens')
    plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')
    plt.setp(ax1.get_yticklabels(), rotation=0)
    
    # 2. CLS token attention (what the model focuses on for classification)
    cls_attention = attention_matrix[0, 1:]  # CLS token attention to other tokens (skip self-attention)
    tokens_for_cls = display_tokens[1:]  # Skip CLS token
    
    # Sort by attention weight
    token_attention_pairs = list(zip(tokens_for_cls, cls_attention))
    token_attention_pairs.sort(key=lambda x: x[1], reverse=True)
    
    # Take top 15 for readability
    top_tokens, top_weights = zip(*token_attention_pairs[:15])
    
    bars = ax2.barh(range(len(top_tokens)), top_weights, color='skyblue')
    ax2.set_yticks(range(len(top_tokens)))
    ax2.set_yticklabels(top_tokens)
    ax2.set_xlabel('Attention Weight')
    ax2.set_title(f'Top Attended Tokens for Classification\n(CLS token attention)', fontsize=14, weight='bold')
    ax2.invert_yaxis()
    
    # Add value labels on bars
    for i, (bar, weight) in enumerate(zip(bars, top_weights)):
        ax2.text(weight + 0.001, i, f'{weight:.3f}', 
                va='center', ha='left', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"📊 Attention Statistics:")
    print(f"   • Number of layers: {len(attentions)}")
    print(f"   • Number of heads per layer: {attentions[0].shape[2]}")
    print(f"   • Sequence length: {len(tokens)}")
    print(f"   • Max attention weight: {attention_matrix.max():.4f}")
    print(f"   • Average attention weight: {attention_matrix.mean():.4f}")
    
    print(f"\n🎯 Top 5 tokens by CLS attention:")
    for i, (token, weight) in enumerate(token_attention_pairs[:5]):
        if token not in ['[SEP]', '[PAD]']:
            print(f"   {i+1}. '{token}': {weight:.4f}")
    
    print(f"👁️ Custom attention visualization complete for: {pred_label}")

print("✅ Attention visualization ready")

✅ Attention visualization ready


### 4.4 GradCAM Implementation

In [10]:
class ModelWrapper(torch.nn.Module):
    """Wrapper to fix SequenceClassifierOutput error with Captum"""
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, input_ids, attention_mask=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.logits

def explain_with_gradcam(text, target_class=None):
    """Generate GradCAM explanation for text"""
    print("⏳ Computing GradCAM attributions...")
    
    try:
        # Tokenize input
        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        
        # Get prediction if not specified
        if target_class is None:
            with torch.no_grad():
                outputs = pt_model(**inputs)
                target_class = torch.argmax(outputs.logits, dim=1).item()
        
        # Ensure target_class is the right type for Captum
        target_class = int(target_class)  # Convert to Python int from numpy.int64
        
        # Use ModelWrapper for Captum compatibility
        wrapped_model = ModelWrapper(pt_model)
        wrapped_model.eval()
        
        # Try to access embedding layer with different paths
        embedding_layer = None
        try:
            embedding_layer = pt_model.bert.embeddings.word_embeddings
        except AttributeError:
            try:
                embedding_layer = pt_model.embeddings.word_embeddings
            except AttributeError:
                try:
                    embedding_layer = pt_model.get_input_embeddings()
                except AttributeError:
                    print("❌ Could not access embedding layer")
                    return
        
        if embedding_layer is None:
            print("❌ Embedding layer not found")
            return
        
        # Initialize LayerGradCam
        layer_gradcam = LayerGradCam(wrapped_model, embedding_layer)
        
        # Generate attributions
        attributions = layer_gradcam.attribute(
            input_ids,
            target=target_class,
            additional_forward_args=(attention_mask,)
        )
        
        # Process attributions
        attribution_scores = attributions.squeeze().detach().cpu().numpy()
        tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze())
        
        if len(attribution_scores.shape) > 1:
            attribution_scores = attribution_scores.sum(axis=-1)
        
        # Visualize
        _visualize_gradcam(tokens, attribution_scores, attention_mask, target_class)
        
        pred_label = label_encoder.inverse_transform([target_class])[0]
        print(f"🌡️ GradCAM explanation for class: {pred_label}")
        
    except Exception as e:
        print(f"❌ GradCAM error: {str(e)}")
        print("💡 Falling back to attention-based attribution...")
        
        # Fallback: Use attention weights as pseudo-GradCAM
        try:
            inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
            with torch.no_grad():
                outputs = pt_model(**inputs, output_attentions=True)
                attentions = outputs.attentions
                
                if attentions is not None and len(attentions) > 0:
                    # Average attention across layers and heads
                    avg_attention = torch.stack(attentions).mean(dim=0).mean(dim=1)
                    cls_attention = avg_attention[0, 0, :].detach().cpu().numpy()
                    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].squeeze())
                    
                    # Visualize as pseudo-GradCAM
                    _visualize_gradcam(tokens, cls_attention, inputs['attention_mask'], 
                                     torch.argmax(outputs.logits, dim=-1).item())
                    print("📊 Used attention weights as attribution fallback")
                else:
                    print("❌ No attention weights available for fallback")
        except Exception as fallback_error:
            print(f"❌ Fallback also failed: {fallback_error}")
            print("💡 Try using SHAP or LIME for alternative explanations")

def _visualize_gradcam(tokens, attribution_scores, attention_mask, target_class):
    """Create GradCAM visualization"""
    fig, ax = plt.subplots(figsize=(14, 4))
    
    # Normalize attributions
    abs_attributions = np.abs(attribution_scores)
    if abs_attributions.max() > 0:
        normalized_attrs = abs_attributions / abs_attributions.max()
    else:
        normalized_attrs = abs_attributions
    
    # Plot tokens with color intensity
    colors = plt.cm.Reds(normalized_attrs)
    x_positions = []
    
    for i, (token, attr, color) in enumerate(zip(tokens, normalized_attrs, colors)):
        if token in ['[CLS]', '[SEP]', '[PAD]'] or attention_mask[0][i].item() == 0:
            continue
        
        clean_token = token.replace('##', '')
        if not clean_token.strip():
            continue
        
        x_pos = len(x_positions) * 1.2
        x_positions.append(x_pos)
        
        bbox_props = dict(boxstyle="round,pad=0.3", facecolor=color, alpha=0.8)
        ax.text(x_pos, 0.5, clean_token, fontsize=11, ha='center', va='center',
                bbox=bbox_props, weight='bold' if attr > 0.5 else 'normal')
    
    # Format plot
    if x_positions:
        ax.set_xlim(-0.5, max(x_positions) + 0.5)
    ax.set_ylim(0, 1)
    ax.axis('off')
    
    pred_label = label_encoder.inverse_transform([target_class])[0]
    ax.set_title(f'GradCAM Attribution for Class: {pred_label}\n(Darker Red = Higher Attribution)', 
                fontsize=14, pad=30, weight='bold')
    
    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap=plt.cm.Reds, norm=plt.Normalize(vmin=0, vmax=1))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', shrink=0.6, pad=0.15)
    cbar.set_label('Attribution Intensity', fontsize=12)
    
    plt.tight_layout()
    plt.show()

print("✅ GradCAM implementation ready")

✅ GradCAM implementation ready


## 5. 🎛️ Interactive Dashboard

In [11]:
class ExplainabilityDashboard:
    """Interactive dashboard for model explainability analysis"""
    
    def __init__(self):
        self.setup_data()
        self.create_interface()
    
    def setup_data(self):
        """Setup data for mistake analysis"""
        predictions_encoded = predict_class(test_texts)
        self.incorrect_indices = np.where(predictions_encoded != true_labels_encoded)[0]
        print(f"📊 Found {len(self.incorrect_indices)} mistakes out of {len(test_texts)} samples")
    
    def create_interface(self):
        """Create the dashboard interface"""
        # Input mode selector
        self.input_mode = widgets.ToggleButtons(
            options=[('Analyze Mistakes', 'mistakes'), ('Custom Text', 'custom')],
            value='mistakes',
            description='Analysis Mode:',
            style={'description_width': 'initial'}
        )
        
        # Mistake selector
        mistake_options = [(f"Mistake {i+1}: {test_texts[idx][:50]}...", i) 
                          for i, idx in enumerate(self.incorrect_indices[:20])]  # Limit for performance
        self.mistake_selector = widgets.Dropdown(
            options=mistake_options,
            description='Select Mistake:',
            style={'description_width': 'initial'}
        )
        
        # Custom text input
        self.text_input = widgets.Textarea(
            placeholder='Enter financial text to analyze...',
            description='Text:',
            layout=widgets.Layout(width='100%', height='80px'),
            style={'description_width': 'initial'}
        )
        
        # Control buttons
        self.analyze_button = widgets.Button(
            description='🚀 Analyze',
            button_style='primary',
            layout=widgets.Layout(width='120px')
        )
        
        self.clear_button = widgets.Button(
            description='🧹 Clear',
            button_style='warning',
            layout=widgets.Layout(width='120px')
        )
        
        # Output tabs
        self.output_tabs = widgets.Tab()
        self.method_outputs = {
            'SHAP': widgets.Output(),
            'LIME': widgets.Output(),
            'Attention': widgets.Output(),
            'GradCAM': widgets.Output()
        }
        
        self.output_tabs.children = list(self.method_outputs.values())
        for i, method in enumerate(self.method_outputs.keys()):
            self.output_tabs.set_title(i, f'{method}')
        
        # Status output
        self.status_output = widgets.Output()
        
        # Event handlers
        self.input_mode.observe(self.on_mode_change, names='value')
        self.analyze_button.on_click(self.on_analyze)
        self.clear_button.on_click(self.on_clear)
    
    def on_mode_change(self, change):
        """Handle input mode change"""
        # Update the input container dynamically
        if hasattr(self, 'input_container'):
            if change['new'] == 'mistakes':
                self.input_container.children = [self.input_mode, self.mistake_selector]
            else:
                self.input_container.children = [self.input_mode, self.text_input]
    
    def update_interface(self):
        """Update interface based on mode"""
        # This method is called by on_mode_change
        pass
    
    def on_analyze(self, button):
        """Handle analyze button click"""
        try:
            # Get text and prediction info
            if self.input_mode.value == 'mistakes':
                mistake_idx = self.mistake_selector.value
                sample_idx = self.incorrect_indices[mistake_idx]
                text = test_texts[sample_idx]
                true_label = label_encoder.inverse_transform([true_labels_encoded[sample_idx]])[0]
                pred_class = int(predict_class(text)[0])  # Ensure Python int
                pred_label = label_encoder.inverse_transform([pred_class])[0]
            else:
                text = self.text_input.value.strip()
                if not text:
                    with self.status_output:
                        clear_output(wait=True)
                        print("❌ Please enter some text to analyze!")
                    return
                pred_class = int(predict_class(text)[0])  # Ensure Python int
                pred_label = label_encoder.inverse_transform([pred_class])[0]
                true_label = "Unknown"
            
            # Generate explanations
            self.generate_explanations(text, pred_label, true_label, pred_class)
            
        except Exception as e:
            with self.status_output:
                clear_output(wait=True)
                print(f"❌ Error during analysis: {str(e)}")
    
    def generate_explanations(self, text, pred_label, true_label, pred_class):
        """Generate all explanations for the text"""
        # Clear outputs
        for output in self.method_outputs.values():
            with output:
                clear_output()
        
        # Create header
        header_html = f"""
        <div style='background: #f8f9fa; padding: 15px; margin: 10px 0; border-radius: 8px; 
                    border-left: 4px solid #007bff; box-shadow: 0 2px 8px rgba(0,0,0,0.1);'>
            <h4 style='margin: 0 0 10px 0; color: #007bff;'>📝 Analysis Summary</h4>
            <p style='margin: 5px 0;'><strong>Text:</strong> <em>"{text}"</em></p>
            <p style='margin: 5px 0;'><strong>Model Prediction:</strong> 
               <span style='color: #28a745; font-weight: bold;'>{pred_label}</span></p>
            {f'<p style="margin: 5px 0;"><strong>True Label:</strong> <span style="color: #dc3545; font-weight: bold;">{true_label}</span></p>' if true_label != "Unknown" else ''}
        </div>
        """
        
        with self.status_output:
            clear_output(wait=True)
            print("🧠 Generating explanations...")
        
        # SHAP
        with self.method_outputs['SHAP']:
            display(HTML(header_html))
            try:
                explain_with_shap(text, pred_class)
            except Exception as e:
                print(f"❌ SHAP failed: {str(e)}")
        
        # LIME
        with self.method_outputs['LIME']:
            display(HTML(header_html))
            try:
                explain_with_lime(text)
            except Exception as e:
                print(f"❌ LIME failed: {str(e)}")
                print("💡 Common LIME issues:")
                print("   - Text preprocessing differences")
                print("   - Prediction function format mismatch")
                print("   - Try using SHAP instead")
        
        # Attention
        with self.method_outputs['Attention']:
            display(HTML(header_html))
            try:
                # BertViz doesn't work well in widget contexts, so use custom visualization
                print("⏳ Generating attention visualization...")
                print("💡 Using custom heatmap (BertViz widgets don't render in dashboard)")
                
                # Get attention data
                inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
                with torch.no_grad():
                    # Force eager attention for compatibility
                    original_impl = getattr(pt_model.config, '_attn_implementation', None)
                    pt_model.config._attn_implementation = 'eager'
                    
                    outputs = pt_model(**inputs, output_attentions=True)
                    attentions = outputs.attentions
                    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
                    
                    # Restore original implementation
                    if original_impl is not None:
                        pt_model.config._attn_implementation = original_impl
                
                if attentions is not None and len(attentions) > 0:
                    pred_label = label_encoder.inverse_transform([pred_class])[0]
                    _visualize_attention_heatmap(attentions, tokens, pred_label)
                else:
                    print("❌ No attention weights available")
                    
            except Exception as e:
                print(f"❌ Attention failed: {str(e)}")
                print("💡 Common Attention issues:")
                print("   - BertViz compatibility with model architecture")
                print("   - JavaScript widget display problems")
                print("   - Try refreshing the notebook kernel")
        
        # GradCAM
        with self.method_outputs['GradCAM']:
            display(HTML(header_html))
            try:
                explain_with_gradcam(text, pred_class)
            except Exception as e:
                print(f"❌ GradCAM failed: {str(e)}")
                print("💡 Common GradCAM issues:")
                print("   - Model architecture compatibility")
                print("   - Captum version mismatch")
                print("   - GPU/CPU tensor issues")
        
        with self.status_output:
            clear_output(wait=True)
            print("✅ Analysis complete! Explore the tabs above.")
    
    def on_clear(self, button):
        """Clear all outputs"""
        for output in self.method_outputs.values():
            with output:
                clear_output()
        
        with self.status_output:
            clear_output(wait=True)
            print("🧹 All results cleared! Ready for new analysis.")
    
    def display(self):
        """Display the dashboard"""
        # Title
        title = widgets.HTML(
            value="""
            <div style='text-align: center; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                        color: white; padding: 20px; border-radius: 10px; margin-bottom: 20px;'>
                <h2 style='margin: 0; font-size: 24px;'>🧠 Financial Sentiment Explainability Dashboard</h2>
                <p style='margin: 10px 0 0 0; opacity: 0.9;'>Comprehensive AI model explanation and analysis</p>
            </div>
            """
        )
        
        # Dynamic input section that updates based on mode
        self.input_container = widgets.VBox([
            self.input_mode,
            self.mistake_selector if self.input_mode.value == 'mistakes' else self.text_input
        ])
        
        # Controls
        controls = widgets.HBox([
            self.analyze_button,
            self.clear_button
        ], layout=widgets.Layout(justify_content='space-between', width='250px'))
        
        # Main dashboard
        dashboard = widgets.VBox([
            title,
            self.input_container,
            controls,
            self.status_output,
            self.output_tabs
        ])
        
        return dashboard

print("✅ Dashboard class defined")

✅ Dashboard class defined


## 6. 🚀 Launch Dashboard

In [12]:
# Initialize and display dashboard
print("🎯 Initializing Explainability Dashboard...")
dashboard = ExplainabilityDashboard()
display(dashboard.display())
print("✅ Dashboard is ready! Use the interface above to analyze model predictions.")

🎯 Initializing Explainability Dashboard...
📊 Found 253 mistakes out of 1212 samples
📊 Found 253 mistakes out of 1212 samples


✅ Dashboard is ready! Use the interface above to analyze model predictions.


## 7. ? Quick Misclassification Analysis

Simple analysis to identify patterns for fine-tuning in the next notebook.

In [15]:
# Quick misclassification analysis
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import json

# Get model predictions on test set
test_predictions = predict_class(test_texts)

# Get misclassified examples
misclassified_mask = test_predictions != true_labels_encoded
misclassified_texts = [test_texts[i] for i in range(len(test_texts)) if misclassified_mask[i]]
misclassified_true = true_labels_encoded[misclassified_mask]
misclassified_pred = test_predictions[misclassified_mask]

print(f"📊 Total misclassifications: {len(misclassified_texts)}")
print(f"📈 Error rate: {len(misclassified_texts)/len(test_texts)*100:.1f}%")

# 1. Confusion patterns
print("\n🔄 Top Confusion Patterns:")
confusion_data = defaultdict(int)
for true_idx, pred_idx in zip(misclassified_true, misclassified_pred):
    true_label = label_encoder.inverse_transform([true_idx])[0]
    pred_label = label_encoder.inverse_transform([pred_idx])[0]
    confusion_data[(true_label, pred_label)] += 1

for (true_label, pred_label), count in sorted(confusion_data.items(), key=lambda x: x[1], reverse=True)[:3]:
    percentage = count / len(misclassified_texts) * 100
    print(f"  {true_label} → {pred_label}: {count} cases ({percentage:.1f}%)")

# 2. Problematic keywords
print("\n🔍 Problematic Keywords:")
correctly_classified_texts = [test_texts[i] for i in range(len(test_texts)) if not misclassified_mask[i]]

vectorizer = TfidfVectorizer(max_features=200, stop_words='english', ngram_range=(1, 2))
all_texts = misclassified_texts + correctly_classified_texts[:len(misclassified_texts)]
vectorizer.fit(all_texts)

misc_tfidf = vectorizer.transform(misclassified_texts).mean(axis=0).A1
correct_tfidf = vectorizer.transform(correctly_classified_texts[:len(misclassified_texts)]).mean(axis=0).A1

feature_names = vectorizer.get_feature_names_out()
score_diff = misc_tfidf - correct_tfidf
top_indices = score_diff.argsort()[-10:][::-1]

problematic_keywords = [(feature_names[i], float(score_diff[i])) for i in top_indices if score_diff[i] > 0.001]
for keyword, score in problematic_keywords[:5]:
    print(f"  {keyword}: {score:.4f}")

# Save results for fine-tuning notebook
os.makedirs('analysis_results', exist_ok=True)
results = {
    'confusion_patterns': {f"{true_label} -> {pred_label}": count for (true_label, pred_label), count in confusion_data.items()},
    'problematic_keywords': problematic_keywords,
    'total_errors': len(misclassified_texts),
    'error_rate': len(misclassified_texts)/len(test_texts)*100
}

with open('analysis_results/misclassification_analysis.json', 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n💾 Results saved to: analysis_results/misclassification_analysis.json")
print(f"📋 Ready for fine-tuning in next notebook!")

📊 Total misclassifications: 253
📈 Error rate: 20.9%

🔄 Top Confusion Patterns:
  neutral → positive: 84 cases (33.2%)
  positive → neutral: 78 cases (30.8%)
  neutral → negative: 38 cases (15.0%)

🔍 Problematic Keywords:
  solutions: 0.0176
  pct: 0.0149
  new: 0.0143
  compared: 0.0134
  oyj: 0.0121

💾 Results saved to: analysis_results/misclassification_analysis.json
📋 Ready for fine-tuning in next notebook!


## 8. 📋 Summary

### ✅ Completed:
- **Interactive Dashboard**: SHAP and LIME explanations for any text
- **Mistake Analysis**: Analyze specific model errors  
- **Misclassification Patterns**: Key insights for fine-tuning

### 📊 Key Findings:
- Error rate: ~20% on test data
- Main confusion patterns identified
- Problematic keywords extracted

### 🔜 Next Steps:
Results saved to `analysis_results/` for **Notebook 6: Fine-tuning with Pruning Methods**